<a href="https://colab.research.google.com/github/gulabpatel/Table_Detection/blob/main/06%3A%20PDF_to_Excel_using_Paddle_OCR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Code walk through video : https://www.youtube.com/watch?v=HZh31OGiQRQ

In [ ]:
from PIL import Image
import cv2
import numpy as np
import pandas as pd 
import tensorflow as tf

# pdf2image

## Installation

In [ ]:
!pip install pdf2image
!apt-get install poppler-utils

## Conversion

In [ ]:
from pdf2image import convert_from_path

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!wget "https://onedrive.live.com/download?cid=DBD24D3DC9712F7A&resid=DBD24D3DC9712F7A%2110213&authkey=AKAJrobqemGhan8" -O data.zip
!unzip -qq data.zip

In [ ]:
images = convert_from_path('/content/drive/MyDrive/documents/BScI.pdf')
# images = cv2.imread("/content/pages/page2.jpg")

In [ ]:
!mkdir pages

In [ ]:
for i in range(len(images)):
  images[i].save('pages/page'+str(i)+'.jpg', 'JPEG')

# Layout

## Installation

In [ ]:
!python3 -m pip install paddlepaddle-gpu
!pip install "paddleocr>=2.0.1"
!pip install protobuf==3.20.0
!git clone https://github.com/PaddlePaddle/PaddleOCR.git

remote: Total 38584 (delta 337), reused 480 (delta 296), pack-reused 38024
Receiving objects: 100% (38584/38584), 320.51 MiB | 32.24 MiB/s, done.
Resolving deltas: 100% (26847/26847), done.
Checking out files: 100% (1277/1277), done.


In [ ]:
!wget https://paddleocr.bj.bcebos.com/whl/layoutparser-0.0.0-py3-none-any.whl
!pip install -U layoutparser-0.0.0-py3-none-any.whl

## Table Extraction

In [ ]:
import cv2
import layoutparser as lp
image = cv2.imread("/content/pages/page0.jpg")
image = image[..., ::-1]

# load model
model = lp.PaddleDetectionLayoutModel(config_path="lp://PubLayNet/ppyolov2_r50vd_dcn_365e_publaynet/config",
                                threshold=0.5,
                                label_map={0: "Text", 1: "Title", 2: "List", 3:"Table", 4:"Figure"},
                                enforce_cpu=False,
                                enable_mkldnn=True)#math kernel library
# detect
layout = model.detect(image)

In [ ]:
layout

In [ ]:
for l in layout:
  # if l.type == 'Table':
    x_1 = int(l.block.x_1)
    y_1 = int(l.block.y_1)
    x_2 = int(l.block.x_2)
    y_2 = int(l.block.y_2)
    
    break

In [ ]:
print(x_1,y_1,x_2,y_2)

In [ ]:
im = cv2.imread('/content/pages/page0.jpg')

In [ ]:
cv2.imwrite('ext_im.jpg', im[y_1:y_2,x_1:x_2])

# Text Detection and Recognition

In [ ]:
from paddleocr import PaddleOCR, draw_ocr

In [ ]:
ocr = PaddleOCR(lang='en')
image_path = '/content/ext_im.jpg'
image_cv = cv2.imread(image_path)
image_height = image_cv.shape[0]
image_width = image_cv.shape[1]
output = ocr.ocr(image_path)

In [ ]:
print(output)

In [ ]:
boxes = [line[0] for line in output]
texts = [line[1][0] for line in output]
probabilities = [line[1][1] for line in output]

In [ ]:
image_boxes = image_cv.copy()

In [ ]:
for box,text in zip(boxes,texts):
  cv2.rectangle(image_boxes, (int(box[0][0]),int(box[0][1])), (int(box[2][0]),int(box[2][1])),(0,0,255),1)
  cv2.putText(image_boxes, text,(int(box[0][0]),int(box[0][1])),cv2.FONT_HERSHEY_SIMPLEX,1,(222,0,0),1)

In [ ]:
cv2.imwrite('detections.jpg', image_boxes)

# Reconstruction

## Get Horizontal and Vertical Lines

In [ ]:
im = image_cv.copy()

In [ ]:
horiz_boxes = []
vert_boxes = []

for box in boxes:
  x_h, x_v = 0,int(box[0][0])
  y_h, y_v = int(box[0][1]),0
  width_h,width_v = image_width, int(box[2][0]-box[0][0])
  height_h,height_v = int(box[2][1]-box[0][1]),image_height

  horiz_boxes.append([x_h,y_h,x_h+width_h,y_h+height_h])
  vert_boxes.append([x_v,y_v,x_v+width_v,y_v+height_v])

  cv2.rectangle(im,(x_h,y_h), (x_h+width_h,y_h+height_h),(0,0,255),1)
  cv2.rectangle(im,(x_v,y_v), (x_v+width_v,y_v+height_v),(0,255,0),1)
  

In [ ]:
cv2.imwrite('horiz_vert.jpg',im)

## Non-Max Suppression

In [ ]:
horiz_out = tf.image.non_max_suppression(
    horiz_boxes,
    probabilities,
    max_output_size = 1000,
    iou_threshold=0.1,
    score_threshold=float('-inf'),
    name=None
)

In [ ]:
horiz_lines = np.sort(np.array(horiz_out))
print(horiz_lines)

In [ ]:
im_nms = image_cv.copy()

In [ ]:
for val in horiz_lines:
  cv2.rectangle(im_nms, (int(horiz_boxes[val][0]),int(horiz_boxes[val][1])), (int(horiz_boxes[val][2]),int(horiz_boxes[val][3])),(0,0,255),1)
  

In [ ]:
cv2.imwrite('im_nms.jpg',im_nms)

In [ ]:
vert_out = tf.image.non_max_suppression(
    vert_boxes,
    probabilities,
    max_output_size = 1000,
    iou_threshold=0.1,
    score_threshold=float('-inf'),
    name=None
)

In [ ]:
print(vert_out)

In [ ]:
vert_lines = np.sort(np.array(vert_out))
print(vert_lines)

In [ ]:
for val in vert_lines:
  cv2.rectangle(im_nms, (int(vert_boxes[val][0]),int(vert_boxes[val][1])), (int(vert_boxes[val][2]),int(vert_boxes[val][3])),(255,0,0),1)
  

In [ ]:
cv2.imwrite('im_nms.jpg',im_nms)

## Convert to CSV

In [ ]:
out_array = [["" for i in range(len(vert_lines))] for j in range(len(horiz_lines))]
print(np.array(out_array).shape)
print(out_array)

In [ ]:
unordered_boxes = []

for i in vert_lines:
  print(vert_boxes[i])
  unordered_boxes.append(vert_boxes[i][0])

In [ ]:
ordered_boxes = np.argsort(unordered_boxes)
print(ordered_boxes)

In [ ]:
def intersection(box_1, box_2):
  return [box_2[0], box_1[1],box_2[2], box_1[3]]

In [ ]:
def iou(box_1, box_2):

  x_1 = max(box_1[0], box_2[0])
  y_1 = max(box_1[1], box_2[1])
  x_2 = min(box_1[2], box_2[2])
  y_2 = min(box_1[3], box_2[3])

  inter = abs(max((x_2 - x_1, 0)) * max((y_2 - y_1), 0))
  if inter == 0:
      return 0
      
  box_1_area = abs((box_1[2] - box_1[0]) * (box_1[3] - box_1[1]))
  box_2_area = abs((box_2[2] - box_2[0]) * (box_2[3] - box_2[1]))
  
  return inter / float(box_1_area + box_2_area - inter)

In [ ]:
for i in range(len(horiz_lines)):
  for j in range(len(vert_lines)):
    resultant = intersection(horiz_boxes[horiz_lines[i]], vert_boxes[vert_lines[ordered_boxes[j]]] )

    for b in range(len(boxes)):
      the_box = [boxes[b][0][0],boxes[b][0][1],boxes[b][2][0],boxes[b][2][1]]
      if(iou(resultant,the_box)>0.1):
        out_array[i][j] = texts[b]

In [ ]:
out_array

In [ ]:
out_array = np.array(out_array)

In [ ]:
pd.DataFrame(out_array).to_csv('sample.csv')

# Convert to owl format

In [ ]:
owl_head = [
  '@prefix : <http://federated.amecse-conferences.org/foodOntology#> .',
  '@prefix owl: <http://www.w3.org/2002/07/owl#> .',
  '@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .',
  '@prefix xml: <http://www.w3.org/XML/1998/namespace> .',
  '@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .',
  '@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .',
  '@base <http://federated.amecse-conferences.org/foodOntology> .',
  '',
  '<http://federated.amecse-conferences.org/foodOntology> rdf:type owl:Ontology .',
  '',
  '',
  ]


In [ ]:
owl_tail = [
  '\n#################################################################',
  '#    Individuals',
  '#################################################################',
  '',
  '###  http://federated.amecse-conferences.org/foodOntology#Azanzi',
  ':Azanzi rdf:type owl:NamedIndividual ,',
  '                 :Person ;',
  '        :name "Azanzi Jiomekong"^^xsd:string .',
  '',
  '',
  '###  http://federated.amecse-conferences.org/foodOntology#Azza',
  ':Azza rdf:type owl:NamedIndividual ,',
  '               :Person ;',
  '      :name "Azza"^^xsd:string .',
  '',
  '',
  '###  http://federated.amecse-conferences.org/foodOntology#Etoga',
  ':Etoga rdf:type owl:NamedIndividual ,',
  '                :Person ;',
  '       :name "Etoga Cosmas"^^xsd:string .',
  '',
  '',
  '###  http://federated.amecse-conferences.org/foodOntology#Folefac',
  ':Folefac rdf:type owl:NamedIndividual ,',
  '                  :Person ;',
  '         :name "Folefac Martins"^^xsd:string .',
  '',
  '',
  '###  Generated by the OWL API (version 4.5.9.2019-02-01T07:24:44Z) https://github.com/owlcs/owlapi',
  '',
  ]

In [ ]:
owl_object_properties_head = [
                        
  '#################################################################',
  '#    Object properties',
  '#################################################################',
  '',
  '',
  ]

In [ ]:
owl_object_properties_body = []
for i in range(1,len(out_array)):
  owl_object_properties_body.append([
    '###  http://federated.amecse-conferences.org/foodOntology#'+out_array[i][0].replace(' ','').replace(',',''),
    ':'+out_array[i][0].replace(' ','').replace(',','')+' rdf:type owl:ObjectProperty ;',
    '     rdfs:domain :'+ out_array[i][0].replace(' ','').replace(',','')+' ;',
    '     rdfs:range :'+ out_array[i][1].replace(' ','').replace(',','')+' .',
    '','',
  ])

In [ ]:
owl_classes_head = [
                        
  '\n#################################################################',
  '#    Classes',
  '#################################################################',
  '',
  '',
  ]

In [ ]:
owl_classes_body = [
  '###  http://federated.amecse-conferences.org/foodOntology#FoodComposition',
  ':FoodComposition rdf:type owl:Class .',
  '',
  '\n',
]

In [ ]:
owl_sub_classes_body = []
for i in range(1,len(out_array)):
  owl_sub_classes_body.append([
    '###  http://federated.amecse-conferences.org/foodOntology#'+out_array[i][0].replace(' ','').replace(',',''),
    ':'+out_array[i][0].replace(' ','').replace(',','')+' rdf:type owl:Class ;',
    '           rdfs:subClassOf :FoodComposition .',
    '','',                      
  ])

In [ ]:
with open('/content/template.owl', 'a') as f:
    f.writelines('\n'.join(owl_head))
    f.writelines('\n'.join(owl_object_properties_head))
    f.writelines('\n'.join(list(np.concatenate(owl_object_properties_body).flat)))
    f.writelines('\n'.join(owl_classes_head))
    f.writelines('\n'.join(owl_classes_body))
    f.writelines('\n'.join(list(np.concatenate(owl_sub_classes_body).flat)))
    f.writelines('\n'.join(owl_tail))